In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import re
from tqdm import tqdm

In [ ]:
dirs=["pmc_json","pdf_json"]
docs=[]
counts=0
for d in dirs:
    print(d)
    counts = 0
    for file in tqdm(os.listdir(f"../input/CORD-19-research-challenge/document_parses/{d}")):#What is an f string?
        file_path = f"../input/CORD-19-research-challenge/document_parses/{d}/{file}"
        j = json.load(open(file_path,"rb"))
        #Taking last 7 characters. it removes the 'PMC' appended to the beginning
        #also paperid in pdf_json are guids and hard to plot in the graphs hence the substring
        paper_id = j['paper_id']
        paper_id = paper_id[-7:]
        title = j['metadata']['title']

        try:#sometimes there are no abstracts
            abstract = j['abstract'][0]['text']
        except:
            abstract = ""
            
        full_text = ""
        bib_entries = []
        for text in j['body_text']:
            full_text += text['text']
                
        docs.append([paper_id, title, abstract, full_text])
        #comment this below block if you want to consider all files
        #comment block start
        counts = counts + 1
        if(counts >= 100):
            break
        #comment block end    
df=pd.DataFrame(docs,columns=['paper_id','title','abstract','full_text'])

print(df.shape)
df.head()

In [ ]:


# importing necessary dependencies
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers



In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q

In [ ]:
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz

In [ ]:
! chown -R daemon:daemon elasticsearch-7.6.2

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )

In [ ]:
! sleep 30

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")


In [ ]:

document_store.write_documents(df[['title', 'full_text']].rename(columns={'title':'name','full_text':'text'}).to_dict(orient='records'))

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:

# Alternative:
reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)
# reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True, context_window_size=500)



In [ ]:
finder = Finder(reader, retriever)

In [ ]:
question = "What is the impact of coronavirus on babies?"
number_of_answers_to_fetch = 2

prediction = finder.get_answers(question=question, top_k_retriever=10, top_k_reader=number_of_answers_to_fetch)
print(f"Question: {prediction['question']}")
print("\n")
for i in range(number_of_answers_to_fetch):
    print(f"#{i+1}")
    print(f"Answer: {prediction['answers'][i]['answer']}")
    print(f"Research Paper: {prediction['answers'][i]['meta']['name']}")
    print(f"Context: {prediction['answers'][i]['context']}")
    print('\n\n')